In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import stats
from lib.DataManager import *
from lib.PolicyStats import *
from lib.Gridworld import *
from lib.Mockworld import *
import os
import cma
from cma.constraints_handler import AugmentedLagrangian, PopulationEvaluator
from IPython import display
import pickle

***Set Params***

In [3]:
#---SET PARAMS---
USE_GRIDWORLD = False
USE_PDIS = True
percent_increase = 0.1
num_policies = 50
es_path = "pickles/van/saved-cma-van-pdis-20.pkl"
gamma = 0.95
delta = 0.01 #1 - delta, confidence

***Setup Environment***

In [4]:
num_states = 18
if(USE_GRIDWORLD):
    print("Using Gridworld")
    num_states = 23
num_actions = 4

In [ ]:
path = "data\data.csv"
if(USE_GRIDWORLD):
    path = "data\gridworld_data.csv"

histories = GetHistories(path, gamma)

line 0
line 1000000
line 2000000
line 3000000
line 4000000
line 5000000
line 6000000
line 7000000
line 8000000
line 9000000
line 10000000
line 11000000
line 12000000
line 13000000
line 14000000
line 15000000
line 16000000
line 17000000
line 18000000
line 19000000
line 20000000
line 21000000
line 22000000
line 23000000
line 24000000
line 25000000
line 26000000
line 27000000
line 28000000
line 29000000
line 30000000
line 31000000
line 32000000
line 33000000
line 34000000
line 35000000
line 36000000
line 37000000
line 38000000
line 39000000
line 40000000
line 41000000
line 42000000
line 43000000
line 44000000
line 45000000
line 46000000
line 47000000
line 48000000
line 49000000
line 50000000


In [ ]:
train, test = SplitData(histories)

In [ ]:
avg_exploratory_J = GetAverageReturn(histories)

In [ ]:
target_performance = GetTargetPerformance(USE_GRIDWORLD, avg_exploratory_J, percent_increase)

In [ ]:
exploration_policy = GetPolicy(train, num_states, num_actions, 1000)
print(exploration_policy)

In [ ]:
ISFunc = ImportanceSampling
if(USE_PDIS):
    ISFunc = PDImportanceSampling

In [ ]:
es = pickle.load(open(es_path, "rb"))
print("ES_Convergence : " + str(sum(es.mean**2)))

In [ ]:
def policy_softmax(policy):
    numerators = np.exp(policy)
    return (numerators.T / np.sum(numerators, axis=1)).T

In [ ]:
avg_returns = 0
new_policy = None
theoretical_fails = 0
actual_fails = 0
i = 0
while i < num_policies:
    passed_theoretical = False
    try:
        print("~x~~x~~~x~~~~~x~~~~~x~~~x~~~~~x~~~~x~~")
        print(i)
        solution = es.ask(1)[0].reshape(num_states, num_actions)
        new_policy = policy_softmax(solution)
        ConfirmBounds(False, target_performance, train, test, exploration_policy, gamma, new_policy, ISFunc, delta)
        passed_theoretical = True
        #policy passes theoretic check

        actual_J = -1
        if(USE_GRIDWORLD):
            actual_J = GetGridworldReturn(new_policy, gamma, 100000)
        else:
            actual_J = GetMockworldReturn(new_policy, gamma, 100000)

        ConfirmBounds(True, actual_J, train, test, exploration_policy, gamma, new_policy, ISFunc, delta)
        #policy passes actual check
#         path = "text_policies\\policy" + str(i + 1) + ".txt"
#         WriteSolutionToTxt(path, solution)
#         loaded_solution = LoadSolutionFromTxt(path, num_states, num_actions)
#         assert np.sum(loaded_solution - solution) == 0
        
        avg_returns += actual_J
        i += 1
    except:
        print("___F A I L___")
        if(passed_theoretical):
            actual_fails += 1
        else:
            theoretical_fails += 1

avg_returns /= num_policies
print("Number of Actual Fails : " + str(actual_fails))
print("Number of Theoretical Fails : " + str(theoretical_fails))
print("Average Return : " + str(avg_returns))
print("Target : " + str(target_performance))
print("Confidence : " + str(delta))

In [ ]:
greed_policy = np.argmax(new_policy,axis=1)

if(USE_GRIDWORLD):
    print(np.insert(greed_policy, [12,16],[-1,-1]).reshape((5,5)))
else:
    print(greed_policy[-2:])
    print(greed_policy[:-2].reshape(4,4))